In [1]:
!pip install multion langchain beautifulsoup4  -q


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import multion

multion.login()
#multion.refresh_token

 * Serving Flask app 'multion.multion'
 * Debug mode: off


 * Running on https://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [09/Jul/2023 04:37:52] "GET /callback?code=5b1bf846-d090-4cd3-8759-30cd39ba6163&state=XldfZUcRwuuGci5xEJ0zkvs02N1JlI HTTP/1.1" 200 -


In [3]:
# LANGCHAIN EXAMPLE
from langchain import  OpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType
import os


os.environ['OPENAI_API_KEY'] = "<openai_api_key>"


from langchain.tools import StructuredTool

class MultionAPI:
    def __init__(self):
        self.tabId = None
        self.new_session_count = 0

    def create_session(self, query: str, url: str) -> str:
        """Always the first step to run any activities that can be done using browser.
        The function parameters 'query' and 'url' both are compulsary.
        'query' is the query that you need to perform in the given url.if there is no 'query'set it as open.
        'url' is the base url of a site."""
        # Only create new session once and continue using update session
        if self.new_session_count < 2:
            response = multion.new_session({"input": query,"url": url})
            self.new_session_count +=1
            self.tabId = response['tabId']
            return  response['message']
        else:
            return "Continue using update session"

    def update_session(self, query:str,url:str) -> str:
        """Updates the existing browser session created using create_session with given action and url, used consequently to handle browser activitites after creating one session of browser.
        The function parameters 'query' and 'url' both are compulsary.
        'query' is the query that you need to perform in the given url.if there is no 'query'set it as open.
        'url' is the base url of a site."""
        response = multion.update_session(self.tabId, {"input": query,"url": url})
        return  response['message']

In [41]:
multion_api = MultionAPI()
tool1 = StructuredTool.from_function(multion_api.create_session)
tool2= StructuredTool.from_function(multion_api.update_session)

llm = OpenAI(temperature=0,max_tokens=1000)

# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type.
agent_executor = initialize_agent(
    [tool1, tool2],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    return_intermediate_steps=True,
    verbose = False
)


In [6]:
agent_executor(f"look up the most recent rank for the most friendliest city in america")

running post
running post


{'input': 'look up the most recent rank for the most friendliest city in america',
 'output': 'The most friendly city in America according to the search results is Cincinnati, Ohio, followed by Chicago, Nashville, Key West, Florida, and Tucson, Arizona.',
 'intermediate_steps': [(AgentAction(tool='create_session', tool_input={'query': 'most friendly city in america', 'url': 'https://www.google.com/'}, log='Action:\n```\n{\n  "action": "create_session",\n  "action_input": {\n    "query": "most friendly city in america",\n    "url": "https://www.google.com/"\n  }\n}\n```\n'),
   'I am searching for the most friendly city in America on Google.\n\n'),
  (AgentAction(tool='update_session', tool_input={'query': 'most friendly city in america rank', 'url': 'https://www.google.com/'}, log=' I need to find the most recent rank\nAction:\n```\n{\n  "action": "update_session",\n  "action_input": {\n    "query": "most friendly city in america rank",\n    "url": "https://www.google.com/"\n  }\n}\n``

In [54]:
import re
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
from langchain.llms import OpenAI
from collections import defaultdict
import re
from typing import List
from collections import defaultdict


class Evaluation():
    def __init__(self):
         self.questions=[]
         self.outputs={"predicted":[],"truth":[]}
         self.queries = {"predicted":[],"truth":[]}
         self.scores= defaultdict(list)
         self.templates = {
              
        "query_evaluation":"""You are trying to answer the following question by querying an API:

                > Question: {question}

                The query sequence you know you should be executing against the API is:

                > Query Sequence: {truth_query}

                Is the following predicted query sequence semantically the same (eg likely to produce the same answer)?

                > Predicted Query Sequence: {predict_query}

                Please give the Predicted Query Sequence a grade of either an A, B, C, D, or F, along with an explanation of why. End the evaluation with 'Final Grade: <the letter>'

                > Explanation: Let's think step by step.""",

          "output_evaluation": """You are trying to answer the following question by querying an API:

                    > Question: {question}

                    The API returned a response of:

                    > API result: {api_response}

                    Your response to the user: {answer}

                    Please evaluate the accuracy and utility of your response to the user's original question, conditioned on the information available.
                    Give a letter grade of either an A, B, C, D, or F, along with an explanation of why. End the evaluation with 'Final Grade: <the letter>'

                    > Explanation: Let's think step by step.""",
        
            "question_generation": """MultiOn is personal browser copilot that can use webapps like twitter for tweeting, google search to get data for topic. 
                    
                    Imagine you're a new user trying to use MultiOn , and want to test its features {features} .
                    
                     What are 2 different requests you want MultiOn to perform ? Make sure the requests should be clear and fully closed,
                      such that no more inputs is needed and they are independent from each request.Assume all accounts are already signed in. 
                     example request: Make a tweet tagging MULTION_AI
                    Wants/Questions:
                    1. """}
        
    
    
    def query_sequence(self,response):
        list_sequence = []
        str_sequence = ""
        intermediate_steps = response["intermediate_steps"]
        for i  in range(len(intermediate_steps)):
            step_obj = intermediate_steps[i]
            tool_input = step_obj[0].tool_input
            step_dict = {"action":step_obj[0].tool,"query":tool_input["query"],"url":tool_input["url"]}
            list_sequence.append(step_dict)
            str_sequence = str_sequence +f"{i}. {list_sequence[i]} \n"
        return list_sequence, str_sequence
    

    def generate_questions(self):    
        prompt = PromptTemplate.from_template(self.templates["question_generation"])

        generation_chain = LLMChain(llm=llm, prompt=prompt)

        questions_ = generation_chain.run(features="").split('\n')
        # Strip preceding numeric bullets
        questions = [re.sub(r'^\d+\. ', '', q).strip() for q in questions_]
        return questions
    
    def generate_dataset(self,file_name="generated_dataset.json",questions=None,):
        if(questions==None):
            questions = self.generate_questions()  
        # Data to be written
        generated_dataset = {}
        print(questions)
        for question in questions:
            response = agent_executor(question)
            print(question,"  --- ",response["output"])
            generated_dataset[question]= {"query_sequence":self.query_sequence(response),"output":response["output"]}
        # Serializing json
        json_object = json.dumps(generated_dataset)

        # Writing to sample.json
        with open(file_name, "w") as outfile:
            outfile.write(json_object)
    
    def parse_eval_results(self,results: List[str]) -> List[float]:
        rubric = {"A": 1.0, "B": 0.75, "C": 0.5, "D": 0.25, "F": 0}
        return [rubric[re.search(r"Final Grade: (\w+)", res).group(1)] for res in results]

    def evaluate_query(self,):
        prompt = PromptTemplate.from_template(self.templates["query_evaluation"])
        eval_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)
        request_eval_results = []
        for question, predict_query, truth_query in list(
            zip(self.questions, self.queries["predicted"], self.queries["truth"])
        ):
            eval_output = eval_chain.run(
                question=question,
                truth_query=truth_query,
                predict_query=predict_query,
            )
            request_eval_results.append(eval_output)
        return self.parse_eval_results(request_eval_results)
    
    def evaluate_output(self):
        prompt = PromptTemplate.from_template(self.templates["output_evaluation"])
        eval_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)
        output_eval_results = []
        for question, predict_output, truth_output in list(
            zip(self.questions, self.outputs["predicted"], self.outputs["truth"])
        ):
            eval_output = eval_chain.run(
                question=question,
                api_response=predict_output,
                answer=truth_output,
            )
            output_eval_results.append(eval_output)
        return self.parse_eval_results(output_eval_results)
        
    
    def evaluate_dataset(self,file_name="dataset.json"):
        self.questions=[]
        self.outputs={"predicted":[],"truth":[]}
        self.queries = {"predicted":[],"truth":[]}
        self.scores= defaultdict(list)

        # Opening JSON file
        f = open(file_name)
        # returns JSON object as dict
        data = json.load(f)

        #Unload the serch_texts and truth_query_sequences
        for question,response in data.items():
            query = response["query_sequence"]
            output = response["output"]
            self.questions.append(question)
            self.queries["truth"].append(query)
            self.outputs["truth"].append(output)

        #predict query sequences
        failed_examples = []

        for question in self.questions:
            try:
                response=agent_executor(question)
                self.queries["predicted"].append(self.query_sequence(response))
                self.outputs["predicted"].append(response["output"])
            except Exception as e:
                failed_examples.append({"q": question, "error": e})
                self.scores["completed"].append(0.0)
        self.scores["query_scores"].extend(self.evaluate_query())
        self.scores["output_scores"].extend(self.evaluate_output())
        """ Printing Format!! """
        header = "{:<20}\t{:<10}\t{:<10}\t{:<10}".format("Metric", "Min", "Mean", "Max")
        print(header)
        for metric, metric_scores in self.scores.items():
            mean_scores = (
                sum(metric_scores) / len(metric_scores)
                if len(metric_scores) > 0
                else float("nan")
            )
            row = "{:<20}\t{:<10.2f}\t{:<10.2f}\t{:<10.2f}".format(
                metric, min(metric_scores), mean_scores, max(metric_scores)
            )
            print(row)

        return self.scores,failed_examples




In [43]:
eval = Evaluation()
eval.generate_dataset(file_name="generated_dataset.json")

['Search for "Best restaurants in New York City"', '2.  Post a tweet with the hashtag #MultiOn_AI']
running post
running post
Search for "Best restaurants in New York City"   ---  Here is a list of the best restaurants in New York City according to Time Out: https://www.timeout.com/newyork/restaurants/best-restaurants-in-nyc
running post
running post
2.  Post a tweet with the hashtag #MultiOn_AI   ---  I have posted the tweet with the hashtag #MultiOn_AI


In [57]:
multion_api = MultionAPI()
tool1 = StructuredTool.from_function(multion_api.create_session)
tool2= StructuredTool.from_function(multion_api.update_session)

llm = OpenAI(temperature=0,max_tokens=1000)

# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type.
agent_executor = initialize_agent(
    [tool1, tool2],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    return_intermediate_steps=True,
    verbose = False
)


In [58]:
eval = Evaluation()
scores,failed = eval.evaluate_dataset(file_name="generated_dataset.json")

running post
running post
running post
Metric              	Min       	Mean      	Max       
query_scores        	0.00      	0.00      	0.00      
output_scores       	0.00      	0.50      	1.00      


In [59]:
scores

defaultdict(list, {'query_scores': [0, 0], 'output_scores': [1.0, 0]})

In [60]:
failed

[]